In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from collections import Counter
import copy

## hexadecimal generator

In [5]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [6]:
## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
# one individual to change to multiple
for individual in individuals_sum_dup_no_self_loop:
#     try:
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    ## preprocessing
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
#     except:
#         print(individual)
#         print(edges)
        
#         continue
    try:
        mid_to_weight(dic)
    except:
        print('mid_to_weight', individual)
        print(edges)
        print()
        continue   
    try:    
        out_to_weight(dic)
    except:
        print('out_to_weight', individual)
        print(edges)
        print()    

out_to_weight 3
[('mid0', 'out2', 3.2788646421486343), ('in0', 'mid1', -0.9787273004730658), ('in2', 'out1', 0.939539142377537), ('mid1', 'mid1', -2.1861132305814133), ('in2', 'mid1', 1.0906760262475201), ('in1', 'mid1', 0.23232107431710666), ('in1', 'out0', -0.38052800244162976), ('mid0', 'out1', -1.9724706241416146)]



In [7]:
individuals_sum_dup_no_self_loop[3]

{'0mid2out': ['mid0', 'out2', 3.2788646421486343],
 '0in1mid': ['in0', 'mid1', -0.9787273004730658],
 '2in1out': ['in2', 'out1', 0.939539142377537],
 '1mid1mid': ['mid1', 'mid1', -2.1861132305814133],
 '2in1mid': ['in2', 'mid1', 1.0906760262475201],
 '1in1mid': ['in1', 'mid1', 0.23232107431710666],
 '1in0out': ['in1', 'out0', -0.38052800244162976],
 '0mid1out': ['mid0', 'out1', -1.9724706241416146]}

In [8]:
edges = individuals_sum_dup_no_self_loop[3]
edges = [tuple(edges[i]) for i in edges]
edges

[('mid0', 'out2', 3.2788646421486343),
 ('in0', 'mid1', -0.9787273004730658),
 ('in2', 'out1', 0.939539142377537),
 ('mid1', 'mid1', -2.1861132305814133),
 ('in2', 'mid1', 1.0906760262475201),
 ('in1', 'mid1', 0.23232107431710666),
 ('in1', 'out0', -0.38052800244162976),
 ('mid0', 'out1', -1.9724706241416146)]

In [9]:
dic = weight_sum_preprocessing(edges)
dic

{'mid1': {'in0': -0.9787273004730658,
  'mid1': -2.1861132305814133,
  'in2': 1.0906760262475201,
  'in1': 0.23232107431710666},
 'out0': {'in1': -0.38052800244162976},
 'out1': {'in2': 0.939539142377537, 'mid0': -1.9724706241416146},
 'out2': {'mid0': 3.2788646421486343}}

In [12]:
def mid_to_weight(dic):
    '''weight for clear input ie. no nested weights: 0mid:{0in, 1in} or 0mid:{0mid, 0in}'''
    for key in dic:
        
        if 'mid' in key and isinstance(dic[key], dict):
#             print(key)
            if set(i[:2] for i in dic[key]) == set(['in']):
                
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
                
            elif key in dic[key].keys() and Counter(i[:2] for i in dic[key])['mi'] == 1:
                
#                 print(dic[key])
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
mid_to_weight(dic)
dic

{'mid1': -0.9509717816997041,
 'out0': {'in1': -0.38052800244162976},
 'out1': {'in2': 0.939539142377537, 'mid0': -1.9724706241416146},
 'out2': {'mid0': 3.2788646421486343}}

In [13]:
def out_to_weight(dic):
    '''weight for nested input ie.  0mid:{1mid} or out:{0mid, 0in}
    use 'mid' neuron only if feeded with 'in' neurons'''
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key != key:
                    dic[key][sub_key] += dic[sub_key]

            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key in dic:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
            
out_to_weight(dic)

In [14]:
mid_to_weight(dic)
dic

{'mid1': -0.9509717816997041,
 'out0': -0.36316591991614544,
 'out1': -0.7750813925109867,
 'out2': 0.9971658131325868}

In [ ]:
dic